<a href="https://colab.research.google.com/github/Chauhan-Purushottam/7th_Sem_Project/blob/master/SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive -o nonempty
!ls drive/"colab Notebooks"

CNN  img2.jpg  practice_opencv.ipynb  Project


In [0]:
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import os
import time
import cv2

In [0]:
df = pd.read_csv("drive/colab Notebooks/Project/data/fer2013.csv")

In [10]:
df.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [0]:
image_list = []
for i in range(len(df['pixels'])):
  img = [int(x) for x in df['pixels'][i].split(' ')]
  img1=np.array(img,dtype='uint8').reshape((48,48))
  level = df['emotion'][i]
  dict = {"level":level,"img":img1}
  image_list.append(dict)
 
  


In [0]:
def build_filters():
  filters = []
  ksize = 31
  for theta in np.arange(0, np.pi, np.pi / 16):
      kern = cv2.getGaborKernel((ksize, ksize), 4.0, theta, 10.0, 0.5, 0, ktype=cv2.CV_32F)
      kern /= 1.5*kern.sum()
      filters.append(kern)
  return filters

def process(img, filters):
  mean = []
  var = []
  #skewness = []
  #kurtosis = []
  #accum = np.zeros_like(img)
  for kern in filters:
    accum = np.zeros_like(img)
    fimg = cv2.filter2D(img, cv2.CV_8UC1, kern)
    np.maximum(accum, fimg, accum)
    mean.append(accum.mean())
    var.append(accum.var())
    #skewness.append(scipy.stats.skew(accum))
    #kurtosis.append(scipy.stats.kurtosis(accum))
    
  return (mean+var)


In [0]:
label = []
features = []
for i in range(len(image_list)):
  filters = build_filters()
  res1 = process(image_list[i]["img"], filters)
  features.append(res1)
  label.append(image_list[i]["level"])
  #dict = {"level":image_list[i]["level"],"img":res1}
      
  #feature_img.append(dict)
  
  #cv2_imshow(res1)
  

In [17]:
label = np.array(label)
features = np.array(features)
print(type(features))

<class 'numpy.ndarray'>


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.33, random_state=42)

In [26]:
print(y_test.shape)

(11843,)


In [23]:
from sklearn import svm
clf = svm.SVC(gamma='scale', decision_function_shape='ovr')
clf.fit(X_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [0]:
y_predict= clf.predict(X_test)

In [31]:
from sklearn.metrics import accuracy_score 
accuracy_score(y_test,y_predict)

0.2577894114666892